In [ ]:
import os
import sys
code_paths = {}
code_paths["repo_name"] = "p5-stress-classifier"
code_paths["package_parent_dir"] = "lib"
    

code_paths["repo_path"] = os.getcwd()
base_dir = os.path.basename(code_paths["repo_path"])
while base_dir != code_paths["repo_name"]:
    code_paths["repo_path"] = os.path.dirname(os.path.abspath(code_paths["repo_path"]))
    base_dir = os.path.basename(code_paths["repo_path"])
    
package_dir = os.path.join(code_paths["repo_path"], "src")
sys.path.insert(0, package_dir)
from stresspred import code_paths
for path in code_paths["neurokit2_paths"]:
    sys.path.insert(0, path)

In [ ]:
import neurokit2 as nk

In [ ]:
seed = 42
duration = 120
sampling_rate = 1000
ecg = nk.ecg_simulate(
    duration=duration, length=None, sampling_rate=sampling_rate, noise=0.01, heart_rate=70, heart_rate_std=1, method="ecgsyn", random_state=seed
)
import numpy as np

In [ ]:
import stopit
import time

In [ ]:
ecg_with_missing = ecg.copy()
missing_length = 10
for start in [5]:
    ecg_with_missing[int(sampling_rate*start):int(sampling_rate*(start + missing_length))] = 0

In [1]:
signal = nk.signal_simulate(duration=20, sampling_rate=1000, frequency=1)
peaks_true = nk.signal_findpeaks(signal)["Peaks"]
peaks = np.delete(peaks_true, [5,6,7,8,9,10,15,16,17])  # create gaps (I added more)
peaks = np.sort(np.append(peaks, [1350, 11350, 18350]))  # add artifacts

NameError: name 'nk' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,3))
plt.plot(signal)    
plt.scatter(peaks_true, signal[peaks_true], color="green")
plt.scatter(peaks, signal[peaks], color="red")

plt.show()

In [1]:
interval_maxes = [None, 1.5, 2.0]
max_seconds = 10
for interval_max in interval_maxes:
    print("Interval max: " + str(interval_max))
    with stopit.ThreadingTimeout(max_seconds) as context_manager:

        t0 = time.time()
        peaks_corrected = nk.signal_fixpeaks(peaks=peaks, interval_min=0.5, interval_max=interval_max, method="neurokit")
        t1 = time.time()

    if context_manager.state == context_manager.EXECUTED:
        total = t1-t0
        print("Took " + str(total) + " seconds. EXECUTED")
        print("Peaks corrected: ")
        print(peaks_corrected)
    elif context_manager.state == context_manager.TIMED_OUT:
        print("Took more than " + str(max_seconds) + " seconds. TIMED_OUT")

Interval max: None


NameError: name 'stopit' is not defined

In [67]:
max_seconds = 10
with stopit.ThreadingTimeout(max_seconds) as context_manager:
    
    t0 = time.time()
    peaks_corrected = nk.signal_fixpeaks(peaks=peaks, interval_min=0.5, interval_max=None, method="neurokit")
    t1 = time.time()

if context_manager.state == context_manager.EXECUTED:
    total = t1-t0
    print("Took " + str(total) + " seconds. EXECUTED")
elif context_manager.state == context_manager.TIMED_OUT:
    print("Took more than " + str(max_seconds) + " seconds. TIMED_OUT")

Took 0.0 seconds. EXECUTED


In [68]:
peaks_corrected

array([  250,  1250,  2250,  3250,  4250, 11250, 12250, 13250, 14250,
       18250, 19250], dtype=int64)

In [69]:
max_seconds = 10
with stopit.ThreadingTimeout(max_seconds) as context_manager:
    
    t0 = time.time()
    peaks_corrected = nk.signal_fixpeaks(peaks=peaks, interval_min=0.5, interval_max=2.0, method="neurokit")
    t1 = time.time()

if context_manager.state == context_manager.EXECUTED:
    total = t1-t0
    print("Took " + str(total) + " seconds. EXECUTED")
elif context_manager.state == context_manager.TIMED_OUT:
    print("Took more than " + str(max_seconds) + " seconds. TIMED_OUT")

Took 0.015625953674316406 seconds. EXECUTED


In [70]:
peaks_corrected

array([   250,   1250,   2250,   3250,   4250,   5250,   6250,   7250,
         8250,   9250,  11250,  12250,  13250,  14250,  10250,   3916,
        -2973,  -9232, -14071, -16963, -18557, -19286, -19438, -19206,
       -18718, -18059, -17286, -16437, -15537, -14603, -13647, -12676,
       -11695, -10707,  -9714,  -8718,  -7720,  -6721,  -5721,  -4721,
        -3720,  -2719,  -1718,   -717,    283,   1283,   2282,   3281,
         4280,   5279,   6278,   7277,   8276,   9275,  10274,  11273,
        12272,  13271,  14270,  15269,  16268,  18250,  19250],
      dtype=int64)

In [8]:
def plot_fixpeaks(signal, peaks_true, peaks, **kwargs):
    peaks_corrected = nk.signal_fixpeaks(peaks=peaks, **kwargs)
    plt.figure(figsize=(15,3))
    plt.plot(signal)    
    plt.scatter(peaks_true, signal[peaks_true], color="green")
    plt.scatter(peaks, signal[peaks], color="red")

    plt.show()

In [ ]:
p = multiprocessing.Process(target=plot_fixpeaks, name="fix peaks", kwargs={"signal":signal,
                                                                            "peaks_true":peaks_true,
                                                                            "peaks":peaks, 
                                                                             "interval_min":0.5, 
                                                                             "interval_max":1.5, 
                                                                             "method":"neurokit"}
                           )
p.start()

# Wait 10 seconds for function
time.sleep(10)

# Terminate function
p.terminate()

# Cleanup
p.join()

In [ ]:
p = multiprocessing.Process(target=plot_fixpeaks, name="fix peaks", kwargs={"signal":signal,
                                                                            "peaks_true":peaks_true,
                                                                            "peaks":peaks, 
                                                                             "interval_min":0.5, 
                                                                             "interval_max":1.5, 
                                                                             "method":"neurokit"}
                           )
p.start()

max_seconds = 5
time.sleep(max_seconds)

p.join(max_seconds)
p.terminate(max_)

# If thread is active
if p.is_alive():
    print("Took more than " + str(max_seconds) + " seconds")

    # Terminate foo
    p.terminate()
    p.join()

In [39]:
p = multiprocessing.Process(target=nk.signal_fixpeaks, name="fix peaks", kwargs={"peaks":peaks, 
                                                                                 "interval_min":0.5, 
                                                                                 "interval_max":1.5, 
                                                                                 "method":"neurokit"}
                           )
p.start()

# Wait 10 seconds for function
time.sleep(10)

# Terminate function
p.terminate()

# Cleanup
p.join()

In [41]:
p = multiprocessing.Process(target=nk.signal_fixpeaks, name="fix peaks", kwargs={"peaks":peaks, 
                                                                                 "interval_min":None, 
                                                                                 "interval_max":None, 
                                                                                 "method":"neurokit"}
                           )
p.start()

# Wait 10 seconds for function
time.sleep(10)

# Terminate function
p.terminate()

# Cleanup
p.join()

In [42]:
p

<Process name='fix peaks' pid=1632 parent=11464 stopped exitcode=0>

In [34]:
peaks_corrected = nk.signal_fixpeaks(peaks=peaks, interval_min=0.5, interval_max=1.5, method="neurokit")

KeyboardInterrupt: 